In [1]:
from common import *

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import albumentations
import torch
from torch.utils.data import Dataset


In [ ]:
def get_transforms(image_size):

    transforms_train = albumentations.Compose([
        albumentations.HorizontalFlip(p=0.5),
        albumentations.ImageCompression(quality_lower=99, quality_upper=100),
        albumentations.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=10, border_mode=0, p=0.7),
        albumentations.Resize(image_size, image_size),
        albumentations.Cutout(max_h_size=int(image_size * 0.4), max_w_size=int(image_size * 0.4), num_holes=1, p=0.5),
        albumentations.Normalize()
    ])

    transforms_val = albumentations.Compose([
        albumentations.Resize(image_size, image_size),
        albumentations.Normalize()
    ])

    return transforms_train, transforms_val


In [ ]:
def get_df(kernel_type, data_dir, train_step):

    df = pd.read_csv(data_dir+'/train.csv')

#     if train_step == 0:
#         df_train = pd.read_csv(os.path.join(data_dir, 'train.csv')).drop(columns=['url'])
#     else:
#         cls_81313 = df.landmark_id.unique()
#         df_train = pd.read_csv(os.path.join(data_dir, 'train.csv')).drop(columns=['url']).set_index('landmark_id').loc[cls_81313].reset_index()
        
    df['filepath'] = df['image'].apply(lambda x: os.path.join(data_dir, 'train_images', x))
#     df = df_train.merge(df, on=['id','landmark_id'], how='left')

    label_group2idx = {landmark_id: idx for idx, label_group in enumerate(sorted(df['label_group'].unique()))}
    idx2label_group = {idx: label_group for idx, landmark_id in enumerate(sorted(df['label_group'].unique()))}
    df['label'] = df['label'].map(label_group2idx)

    out_dim = df.label.nunique()
    print('out_dim = ',out_dim)
    return df, out_dim


In [ ]:
import transformers as ppb
model_class, tokenizer_class, pretrained_weights = (ppb.XLMRobertaModel, ppb.XLMRobertaTokenizer, 'xlm-roberta-base')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
class ShopDataset(Dataset):
    def __init__(self, csv, split, mode, transform=None):

        self.csv = csv.reset_index()
        self.text_model_pretrained_weights = 'xlm-roberta-base'
        self.text_model_tokenizer_class = ppb.XLMRobertaTokenizer
        self.split = split
        self.mode = mode
        self.transform = transform

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, index):
        row = self.csv.iloc[index]
        title = row.title
        label = row.label
        image = cv2.imread(filepath)[:,:,::-1]

        if self.transform is not None:
            res = self.transform(image=image)
            image = res['image'].astype(np.float32)
        else:
            image = image.astype(np.float32)

        image = image.transpose(2, 0, 1)
        
        tokenized = tokenizer.encode(title, add_special_tokens=True)
        text_max_len = 61
#         for i in tokenized.values:
#             if len(i) > max_len:
#                 max_len = len(i)
        if len(tokenized) > text_max_len:
            text_max_len = len(tokenized)
        
        padded = np.array([tokenized + [0]*(max_len-len(tokenized))])
        attention_mask = np.where(padded != 0, 1, 0)
        input_ids = torch.tensor(padded)  
        attention_mask = torch.tensor(attention_mask)
        return torch.tensor(image), torch.tensor(label), input_ids, attention_mask
